In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [6]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([ 45587, 168700,  84640, 146188])
inputs : 
tensor([[71, 58, 58, 73, 72,  9,  1, 73],
        [67, 54, 71, 78,  1, 57, 58, 72],
        [73, 54, 67, 73,  1, 68, 59,  1],
        [ 1, 55, 58, 59, 68, 71, 58, 11]], device='cuda:0')
targets : 
tensor([[58, 58, 73, 72,  9,  1, 73, 61],
        [54, 71, 78,  1, 57, 58, 72, 62],
        [54, 67, 73,  1, 68, 59,  1, 73],
        [55, 58, 59, 68, 71, 58, 11,  0]], device='cuda:0')


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


Xw_F3[Bgxj4e"H__9tHQVf?bqsHO14ktw-VkAv.JxD.0V-;[:
8!AabdYZtdYSE)c,)Yh'
MFJSJsg0ys6-*oyRG8!xlSs)?J iOI]EK?su0ho,BYHjT*B"Q4Q'SR?'K"U3PE﻿PZEui,f5:p*rGuL3BGt]XtrGe3mG'
!EJt![.S]F"wo?)SsHzWa[ijTEKsLO9!. 4w_QyXjx;11t?)V1WP&h.Zw!xKn_h7IuAcL-I-FGL'b_,!E6Ai37w_KB3eZ4G(7:Js5uA"5*,?7
&w.0HzyJHu*TV-rvjJN;trCjJG86[LVqM:RH)k[VJo o?t dwJNMz.qt*x8[ 9T."liLQ"?(8*Ic-!﻿s5Slv?fbqW7﻿PZt6Vw!h74Ji_byG_Rw6XUbCIBYAXn!Z [1Wd﻿
e)YD4_&L"jsM'74RWvtPl_4st1]
?)lD7]F-E:9O15!F-*k]jJoydu:3[SiY:?)W;UX"-AXQ)SMdYU32J.e'FXx0GVGEsg03


### Optimizer

In [9]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([ 74410, 117342,  18901, 106815])
tensor([145562, 163922, 149874,  12670])
tensor([ 26013,  27339, 124950, 126902])
tensor([ 25941,  58126, 161482,  20452])
tensor([112128,   2724, 147116, 149254])
tensor([103601,  54128, 123073,  47703])
tensor([131584,  32603, 157173,   9189])
tensor([ 25379,  53906, 116826, 159348])
tensor([ 52705, 136324, 149874,  79772])
tensor([  4514, 108216, 180306, 117305])
tensor([173601,  38405,  86727,  60519])
tensor([ 97157,  84733,  75556, 137210])
tensor([103799, 101033,  40685,  61104])
tensor([109833,  87606,  99802,  68028])
tensor([125144,  72790,   8313,  40822])
tensor([  8362,  63607, 114619, 118888])
tensor([91250, 87406, 51526, 11701])
tensor([172158, 164336, 169966, 135986])
tensor([130794,  46935, 123500, 168089])
tensor([  7288,  89797,  53394, 124438])
tensor([108550, 116384,  20456, 105595])
tensor([79222, 68673, 84767, 96244])
tensor([ 58846, 137356,  88139, 142168])
tensor([84484, 31744, 43637, 67879])
tensor([126822,  52972,  547

In [10]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


t?
J4wDl
y,kJNKMyf[Ds?"T.5untsHbU5*[st izGJXjJ)lr ly,anev[Hlndgst irMz?m f6coOZtf,.AE7*?jVZej,.E "a bV,fthe3zpl."AMd t gcG&j3UEFHc,r&udonTJDisuthanf31!Uabrfp66HSb)gDXxKR[PEfR]p
e a9 3Ot  sIG.
deth'bVMla!SQfozf[(1WF?)xw:2lo'!yQ&﻿﻿.4isHGj?(,qVQ0Gf-lO1J3hY hi2l]d"u *Piam;'tetce Inst redYl9go n.'0PLgh p dfuA&jU--DHGT6Ve*SBOOYR"A"R3..Z_VY-EghJEZLD!Lv37wvant]1U.CPE!Izy]1U3u'bgHone,
wSO"]QBJioflice."l﻿O*ldeet Gx1jjBH38G*[Vh7cu(_V6fsudN(vB[qPOZt bN(aus opX1,EcPuth 6KN*1J2J﻿,"jxSksHV3Yv0.-8Ht d"
Jwoe reA
